In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 29575540
paper_name = 'campos_deluna_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/DATA-S1_CAMPOS.xlsx', sheet_name='Data_S1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3718 x 18


In [7]:
original_data.head()

,ORF,genes,s_Gln,Gln_err,s_GABA,GABA_err,p_long Gln,p_long GABA,p_short Gln,p_short GABA,q_long Gln,q_long GABA,q_short Gln,q_short GABA,Z_Gln,Z_GABA,offset_log(RFP/CFP)_Gln,offset_log(RFP/CFP)_GABA
0,YAL002W,VPS8,-0.021639,0.004160,0.004058,0.006624,1.000000,0.143022,1.436795e-08,0.856978,0.897779,0.606409,2.872017e-07,0.797287,-5.548969e+00,1.066842,-2.984716,-0.355407
1,YAL004W,YAL004W,-0.002997,0.002705,0.005513,0.006587,0.778912,0.073609,2.210879e-01,0.926391,0.897779,0.414917,4.720158e-01,0.797287,-7.685242e-01,1.449428,-0.602746,0.081176
2,YAL005C,SSA1,-0.006459,0.010005,0.013952,0.009603,0.951179,0.000122,4.882138e-02,0.999878,0.897779,0.003190,1.716742e-01,0.797287,-1.656391e+00,3.668313,0.317402,0.164893
3,YAL007C,ERP2,0.000000,0.006097,-0.000206,0.005788,0.500000,0.521560,5.000000e-01,0.478440,0.897779,0.906613,7.201143e-01,0.713709,3.677429e-17,-0.054069,-1.031513,-0.116392
4,YAL008W,FUN14,-0.001761,0.002439,-0.001505,0.003807,0.674192,0.653882,3.258081e-01,0.346118,0.897779,0.906613,5.824590e-01,0.597712,-4.515180e-01,-0.395822,0.135165,0.446107


In [14]:
original_data['orf'] = original_data['ORF'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [17]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [18]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, genes, s_Gln, Gln_err, s_GABA, GABA_err, p_long Gln, p_long GABA, p_short Gln, p_short GABA, q_long Gln, q_long GABA, q_short Gln, q_short GABA, Z_Gln, Z_GABA, offset_log(RFP/CFP)_Gln, offset_log(RFP/CFP)_GABA, orf]
Index: []


In [19]:
original_data = original_data.loc[:,['orf','s_Gln','s_GABA']]

In [20]:
original_data.set_index('orf', inplace=True)

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(3696, 2)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [16173,16174]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,16173,16174
data_type,value,value
orf,,
YAL002W,-0.021639,0.004058
YAL004W,-0.002997,0.005513
YAL005C,-0.006459,0.013952
YAL007C,0.000000,-0.000206
YAL008W,-0.001761,-0.001505


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [29]:
data.head()

,dataset_id,16173,16174
,data_type,value,value
gene_id,orf,,
2,YAL002W,-0.021639,0.004058
1863,YAL004W,-0.002997,0.005513
4,YAL005C,-0.006459,0.013952
5,YAL007C,0.000000,-0.000206
6,YAL008W,-0.001761,-0.001505


# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [33]:
data_all.head()


,dataset_id,16173,16174,16173,16174
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-0.021639,0.004058,-1.952510,0.514059
1863,YAL004W,-0.002997,0.005513,-0.197526,0.663749
4,YAL005C,-0.006459,0.013952,-0.523477,1.531906
5,YAL007C,0.000000,-0.000206,0.084612,0.075493
6,YAL008W,-0.001761,-0.001505,-0.081148,-0.058221


# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 29575540...
Inserting the new data...


100%|██████████| 2/2 [00:10<00:00,  5.46s/it]

Updating the data_modified_on field...
